In [1]:
# Imports required modules
import numba
import numpy as np
from math import pi

from only_head import calculate_head

from lcode2dPy import DiagnosticsFXi, DiagnosticsColormaps, \
                      DiagnosticsTransverse, SaveRunState, Cartesian3dSimulation

In [2]:
# Checks the number of available threads.
print(numba.config.NUMBA_NUM_THREADS) 

# Here you can set the number of threads you want to use.
# Your number must be less than numba.config.NUMBA NUM_THREADS.
numba.set_num_threads(2)

8


In [ ]:
# Sets parameters of test 1:
config = {
    'geometry': '3d',
    'processing-unit-type': 'cpu',
    'window-width-step-size': 0.05,
    'window-width': 16,

    'window-length': 5, # Not including the head!!!
    'xi-step': 0.05,

    'time-limit': 1e-10,
    'time-step': 1e-10,

    'plasma-particles-per-cell': 1,
    'noise-reductor-amplitude': 1
}

# Sets the charge distribution for test 1 beam:
COMPRESS, BOOST, SIGMA, SHIFT = 1, 1, 1, 0
def rho_b_test1(xi, x_grid, y_grid):
    if xi < -2 * np.sqrt(2 * np.pi) / COMPRESS or xi > 0:
        return 0.
        # return np.zeros_like(xs)
    r = np.sqrt(x_grid**2 + (y_grid - SHIFT)**2)
    return (.05 * BOOST * np.exp(-.5 * (r / SIGMA)**2) *
            (1 - np.cos(xi * COMPRESS * np.sqrt(np.pi / 2))))

print('The length of a beam is', 2 * np.sqrt(2 * np.pi) / COMPRESS)

In [ ]:
# You can calculate the head only once.
calculate_head(window_length=5.1, config_dictionary=config, rho_b=rho_b_test1)

In [5]:
# Should work correctly. Compare results with Ez below.
# The numbers should be close enough.

# xi=+0.0000 Ez=-0.0000e+00
# xi=-0.0200 Ez=-2.9321e-07
# xi=-0.0400 Ez=-8.8055e-07
# xi=-0.0600 Ez=-2.0405e-06
# xi=-0.0800 Ez=-4.1982e-06
# xi=-0.1000 Ez=-7.4573e-06
# xi=-0.1200 Ez=-1.2074e-05
# xi=-0.1400 Ez=-1.8043e-05
# xi=-0.1600 Ez=-2.5885e-05
# xi=-0.1800 Ez=-3.5613e-05
# xi=-0.2000 Ez=-4.7634e-05
# xi=-0.2200 Ez=-6.4111e-05
# xi=-0.2400 Ez=-8.4316e-05
# xi=-0.2600 Ez=-1.0699e-04
# xi=-0.2800 Ez=-1.3406e-04
# xi=-0.3000 Ez=-1.6460e-04
# xi=-0.3200 Ez=-1.9717e-04
# xi=-0.3400 Ez=-2.3431e-04
# xi=-0.3600 Ez=-2.7690e-04
# xi=-0.3800 Ez=-3.2427e-04
# xi=-0.4000 Ez=-3.7701e-04
# xi=-0.4200 Ez=-4.3480e-04
# xi=-0.4400 Ez=-4.9688e-04
# xi=-0.4600 Ez=-5.6706e-04
# xi=-0.4800 Ez=-6.4730e-04
# xi=-0.5000 Ez=-7.3259e-04

In [6]:
# Sets diagnostics and their parameters:
diag = [DiagnosticsFXi(
            output_period=0, saving_xi_period=10, f_xi='Ez', f_xi_type='both'),
        DiagnosticsTransverse(
            output_period=0, saving_xi_period=1, colormaps='rho,x_offt,y_offt',
            colormaps_type='both'),
        DiagnosticsColormaps(
            output_period=0, colormaps='Ez, rho'),
        SaveRunState(
            saving_period=0, save_plasma=True)]

# and beam parameters:
beam_parameters = {'beam':{'ampl':0}}

# beam_parameters = {'current': 0.01 * (2*pi), 'particles_in_layer': 2000}

In [7]:
sim = Cartesian3dSimulation(config=config, diagnostics=diag,
                            beam_parameters=beam_parameters)

# Use it if you want to start from a plasmastate:
sim.external_plasmastate = True
sim.path_to_plasmastate = 'plasmastate_after_the_head.npz'

In [ ]:
sim.step()